In [1]:
import pandas
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime, date
from collections import defaultdict, OrderedDict

## Read the CSV, parse the 24/7 and late fields into a single field


In [2]:
restaurants = pandas.read_csv('restaurant_database.csv')

restaurants['open'] = ['24/7' if value == 'yes' else 'early' for value in restaurants['24/7?']]
restaurants['open'] = ['late' if (current == 'early' and open_late == 'yes') else current
                       for current, open_late 
                       in zip(restaurants['open'], restaurants['Open late (after 10pm)'])]

## Define translation functions

We take the knowledge base, as defined in the CSV (downloaded from Google sheets), and define how to translate it to a Prolog KB. This includes what should be the names for conditions which are boolean (not multivalued), various measured of input sanitization, and other renamings.

In [17]:
def sanitze_to_atom(name, prefix='w'):
    sanitized = name.lower().replace(' ', '_').replace('-', '_').replace('?', '').replace(',', '')
    if sanitized[0].isdigit():
        sanitized = prefix + sanitized        
    return sanitized


BOOLEAN_AFFIRMATIVE_NAME = {
    'coffee': 'served',
    'alcohol': 'served',
    'vegeterian': 'friendly',
    'pescaterian': 'friendly',
    'take_away': 'offered'
}


COLUMNS_TO_IGNORE = (
    'Name', 'Address', 'Open late (after 10pm)', '24/7?'
)


TRUE_VALUES = ('yes', 'y', 'True', 'true', True)


class BooleanTransformer:
    def __init__(self, col_atom):
        self.col_atom = col_atom
        
    def __call__(self, value):
        transformed = '{name}({value})'.format(name=self.col_atom, 
                                               value=BOOLEAN_AFFIRMATIVE_NAME[self.col_atom])
                
        if value in TRUE_VALUES:
            return transformed 

        return 'not({t})'.format(t=transformed)

    
class MultivaluedTransformer:
    def __init__(self, col_atom):
        self.col_atom = col_atom
        
    def __call__(self, value):
        return '{name}({value})'.format(name=self.col_atom, 
                                        value=sanitze_to_atom(value))

    
def build_transformers(knowledge_base):
    transformers = {}
    multivalued = []
    
    for column in knowledge_base.columns:
        if column in COLUMNS_TO_IGNORE:
            continue
        
        column_atom = sanitze_to_atom(column)
        if column_atom in BOOLEAN_AFFIRMATIVE_NAME:
            transformers[column] = BooleanTransformer(column_atom)
            
        else:
            transformers[column] = MultivaluedTransformer(column_atom)
            multivalued.append(column)
            
    return transformers, multivalued


INFORMATIVE_FIELDS = ('Name', 'Address', 'Neighborhood')
            
    
def transform_kb(knowledge_base):
    transformers, multivalued = build_transformers(knowledge_base)
    restaurants = {}
    output = []
    
    for rest_id, rest_row in knowledge_base.iterrows():
        transformed_values = [transformers[col](rest_row[col]) for col in transformers]
        output.append('suggest(rest_{id}) :- {cond}.'.format(id=rest_id, 
                                                             cond=', '.join(transformed_values)))
        
        restaurants['rest_{id}'.format(id=rest_id)] = {field: rest_row[field] 
                                                       for field in INFORMATIVE_FIELDS}
    
    output.append('')
    for col in transformers:
        col_atom = sanitze_to_atom(col)
        output.append('{col}(X) :- ask({col}, X).'.format(col=col_atom))
    
    output.append('')
    output.extend(['multivalued({col}).'.format(col=sanitze_to_atom(col)) for col in multivalued])
    
    return output, restaurants
    
        

In [18]:
output, rests = transform_kb(restaurants)

In [25]:
print('\n'.join(output))

suggest(rest_0) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(japanese), alcohol(served), not(take_away(offered)), not(coffee(served)), pescaterian(friendly), open(early), price_range(w7500_15000).
suggest(rest_1) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(korean), not(alcohol(served)), take_away(offered), not(coffee(served)), pescaterian(friendly), open(early), price_range(w5000_7500).
suggest(rest_2) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(taiwanese), alcohol(served), not(take_away(offered)), not(coffee(served)), pescaterian(friendly), open(early), price_range(w15000_25000).
suggest(rest_3) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(korean), not(alcohol(served)), take_away(offered), not(coffee(served)), not(pescaterian(friendly)), open(late), price_range(w5000_7500).
suggest(rest_4) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(korean), alcohol(served), not(take_away(offered)), 

In [26]:
KB = """
% Enter your KB below this line:

:- dynamic(known/3). 

suggest(rest_0) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(japanese), alcohol(served), not(take_away(offered)), not(coffee(served)), pescaterian(friendly), open(early), price_range(w7500_15000).
suggest(rest_1) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(korean), not(alcohol(served)), take_away(offered), not(coffee(served)), pescaterian(friendly), open(early), price_range(w5000_7500).
suggest(rest_2) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(taiwanese), alcohol(served), not(take_away(offered)), not(coffee(served)), pescaterian(friendly), open(early), price_range(w15000_25000).
suggest(rest_3) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(korean), not(alcohol(served)), take_away(offered), not(coffee(served)), not(pescaterian(friendly)), open(late), price_range(w5000_7500).
suggest(rest_4) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(korean), alcohol(served), not(take_away(offered)), not(coffee(served)), not(pescaterian(friendly)), open(late), price_range(w7500_15000).
suggest(rest_5) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(italian), alcohol(served), not(take_away(offered)), coffee(served), pescaterian(friendly), open(late), price_range(w15000_25000).
suggest(rest_6) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(korean), not(alcohol(served)), not(take_away(offered)), coffee(served), not(pescaterian(friendly)), open(early), price_range(w7500_15000).
suggest(rest_7) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(breakfast), alcohol(served), not(take_away(offered)), coffee(served), pescaterian(friendly), open(late), price_range(w7500_15000).
suggest(rest_8) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(vietnamese), alcohol(served), not(take_away(offered)), not(coffee(served)), not(pescaterian(friendly)), open(w24/7), price_range(w7500_15000).
suggest(rest_9) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(vietnamese), alcohol(served), not(take_away(offered)), not(coffee(served)), not(pescaterian(friendly)), open(late), price_range(w7500_15000).
suggest(rest_10) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(korean), not(alcohol(served)), not(take_away(offered)), not(coffee(served)), not(pescaterian(friendly)), open(early), price_range(w7500_15000).
suggest(rest_11) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(japanese), not(alcohol(served)), take_away(offered), not(coffee(served)), not(pescaterian(friendly)), open(w24/7), price_range(w5000_7500).
suggest(rest_12) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(vietnamese), not(alcohol(served)), take_away(offered), not(coffee(served)), not(pescaterian(friendly)), open(early), price_range(w5000_7500).
suggest(rest_13) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(japanese), alcohol(served), not(take_away(offered)), not(coffee(served)), not(pescaterian(friendly)), open(late), price_range(w25000+).
suggest(rest_14) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(american), alcohol(served), take_away(offered), coffee(served), not(pescaterian(friendly)), open(early), price_range(w7500_15000).
suggest(rest_15) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(japanese), alcohol(served), take_away(offered), not(coffee(served)), pescaterian(friendly), open(early), price_range(w7500_15000).
suggest(rest_16) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(japanese), not(alcohol(served)), take_away(offered), not(coffee(served)), pescaterian(friendly), open(early), price_range(w7500_15000).
suggest(rest_17) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(american), not(alcohol(served)), take_away(offered), coffee(served), pescaterian(friendly), open(w24/7), price_range(w7500_15000).
suggest(rest_18) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(american), not(alcohol(served)), take_away(offered), coffee(served), pescaterian(friendly), open(w24/7), price_range(w7500_15000).
suggest(rest_19) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(korean), alcohol(served), take_away(offered), not(coffee(served)), not(pescaterian(friendly)), open(early), price_range(w7500_15000).
suggest(rest_20) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(chinese), alcohol(served), take_away(offered), not(coffee(served)), pescaterian(friendly), open(early), price_range(w7500_15000).
suggest(rest_21) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(korean), alcohol(served), take_away(offered), not(coffee(served)), pescaterian(friendly), open(late), price_range(w7500_15000).
suggest(rest_22) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(korean), alcohol(served), not(take_away(offered)), not(coffee(served)), not(pescaterian(friendly)), open(w24/7), price_range(w7500_15000).
suggest(rest_23) :- neighborhood(itaewon), not(vegeterian(friendly)), cuisine_type(turkish), not(alcohol(served)), take_away(offered), not(coffee(served)), not(pescaterian(friendly)), open(early), price_range(w5000_7500).
suggest(rest_24) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(american), not(alcohol(served)), take_away(offered), coffee(served), pescaterian(friendly), open(late), price_range(w15000_25000).
suggest(rest_25) :- neighborhood(gangnam), not(vegeterian(friendly)), cuisine_type(korean), alcohol(served), not(take_away(offered)), not(coffee(served)), not(pescaterian(friendly)), open(early), price_range(w7500_15000).
suggest(rest_26) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(korean), alcohol(served), take_away(offered), not(coffee(served)), pescaterian(friendly), open(w24/7), price_range(w5000_7500).
suggest(rest_27) :- neighborhood(gangnam), vegeterian(friendly), cuisine_type(japanese), not(alcohol(served)), take_away(offered), not(coffee(served)), pescaterian(friendly), open(early), price_range(w7500_15000).

neighborhood(X) :- ask(neighborhood, X).
vegeterian(X) :- ask(vegeterian, X).
cuisine_type(X) :- ask(cuisine_type, X).
alcohol(X) :- ask(alcohol, X).
take_away(X) :- ask(take_away, X).
coffee(X) :- ask(coffee, X).
pescaterian(X) :- ask(pescaterian, X).
open(X) :- ask(open, X).
price_range(X) :- ask(price_range, X).

multivalued(neighborhood).
multivalued(cuisine_type).
multivalued(price_range).
multivalued(open).

% The code below implements the prompting to ask the user:


% Asking clauses

multivalued(none).

ask(A, V):-
known(y, A, V), % succeed if true
!. % stop looking

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

ask(A, V):-
not(multivalued(A)),
% write_py(A:not_multivalued),
known(y, A, V2),
V \== V2,
!, fail.

ask(A, V):-
read_py(A,V,Y), % get the answer
asserta(known(Y, A, V)), % remember it
Y == y. % succeed or fail
"""

with open("KB_A.pl", "w") as text_file:
    text_file.write(KB)

In [34]:
# The code here will ask the user for input based on the askables
# It will check if the answer is known first

from pyswip.prolog import Prolog
from pyswip.easy import *

prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True


# This dictionary maps a variable name to a string template 
# of how to ask the user about it in a friendly manner
DEFAULT_QUESTION = '{A} is {V}? '
ATOM_TO_QUESTION_MAPPING = defaultdict(lambda: DEFAULT_QUESTION)
ATOM_TO_QUESTION_MAPPING.update({
    'neighborhood': 'Do you wish to eat in the {V} {A} ? ',
    'vegeterian': 'Should the restaurant be {A} {V}? ',
    'pescaterian': 'Should the restaurant be {A} {V}? ',
    'cuisine_type': 'Do you wish to eat {V} food tonight? ',
    'alcohol': 'Do you need a place that serves {A}? ',
    'coffee': 'Do you need a place that serves {A}? ',
    'take_away': 'Do you need a place that offers take-away? ',
    'open': 'Would you like a place that is {A} {V}? ',
    'price_range': 'Are you okay with a price range of {V} per person? '
})
YES_VALUES = ('yes', 'y', 'true', 'sure', 'okay', 'definitely', 'cool')


def read_py(A,V,Y):
    A_value = A.get_value() if type(A) == Atom else str(A)
    V_value = V.get_value() if type(V) == Atom else str(V)
    question = ATOM_TO_QUESTION_MAPPING[A_value]
    
    if A_value == 'price_range':
        V_value = '₩' + V_value[1:].replace('_', '-')
        
    user_input = raw_input(question.format(A=A_value, V=V_value))
    
    if user_input.lower() in YES_VALUES:
        user_input = 'y'
    
    Y.unify(user_input)
    return True


write_py.arity = 1
read_py.arity = 3

registerForeign(read_py)
registerForeign(write_py)

prolog.consult("KB_A.pl") # open the KB
call(retractall(known))
found = False

for soln in prolog.query("suggest(X).", maxresult=1):
    rest = soln['X']
    info = rests[rest]
    print('Your restaurant reccomendation is {name} at {address} in {neighborhood}'.format(
        name=info['Name'], address=info['Address'], neighborhood=info['Neighborhood']))
    
    found = True
    
if not found:
    print('Sorry, no acceptable restaurants found, please try again.')

KeyboardInterrupt: 

PrologError: Caused by: 'suggest(X).'. Returned: 'error(domain_error(foreign_return_value, 140413775152536), context(/(read_py, 3), _3602))'.